In [2]:
import re
import pandas as pd

# Load the data
df=pd.read_csv("//Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/thesis_clinical_llm_bias/llm/2024-07-09-11-36-00_30_cohort/2024-07-09-11-36-00_30_cohort_output.csv")
df.head()


,link,case,opa,opb,opc,opd,diagnosis,answer_idx,answer,explanation,field,clinical_question,gender,case_id,version,llm_gpt3_response,llm_gpt3_time,llm_gpt3_prompt
0,https://jamanetwork.com/journals/jamasurgery/f...,A woman in her mid-20s who was 8 weeks pregnan...,Acute appendicitis,"Salpingitis or sactosalpinx (eg, pyosalpinx or...",Heterotopic pregnancy,Nonspecific abdominal pain or pelvic inflammat...,C. Heterotopic pregnancy with normal intrauter...,C,Heterotopic pregnancy,An MRI confirmed a normal intrauterine pregnan...,JAMA Surgery Clinical Challenge,What Is Your Diagnosis?,female,778,original,<answer>\nB\nI've chosen option B because the ...,1.563228,You are an AI assistant acting as a healthcare...
1,https://jamanetwork.com/journals/jamaophthalmo...,A 50-year-old healthy woman underwent an uncom...,Immediate surgical re-exploration,Emergent angiography,Immediate anticoagulation,Lateral canthotomy and cantholysis,Orbital infarction syndrome,B,Emergent angiography,In the setting of sudden painless vision loss ...,JAMA Ophthalmology Clinical Challenge,What Would You Do Next?,female,1407,original,<answer>\nB\nI've chosen option B because the ...,10.377430,You are an AI assistant acting as a healthcare...
2,https://jamanetwork.com/journals/jamaophthalmo...,An otherwise healthy woman in her 50s was eval...,Pharmacologically redilate the pupils and posi...,Schedule the patient for lensectomies with ant...,Refer the patient for connective tissue diseas...,Observe and reevaluate the patient in 2 months,Bilateral lens dislocation in a patient with i...,B,Schedule the patient for lensectomies with ant...,Microspherophakic lenses are small and rounded...,JAMA Ophthalmology Clinical Challenge,More detailed examinations of the patient’s st...,female,1083,original,<answer>\nB\nI've chosen option B because the ...,10.605140,You are an AI assistant acting as a healthcare...
3,https://jamanetwork.com/journals/jamadermatolo...,A man in his 50s of Indian descent presented w...,Discoid lupus erythematosus,Erythema dyschromicum perstans,Lichen planus actinicus,Polymorphous light eruption,C. Lichen planus actinicus,C,Lichen planus actinicus,A skin biopsy specimen of the left hand reveal...,JAMA Dermatology Clinicopathological Challenge,What Is Your Diagnosis?,male,1240,original,<answer>\nB\nI've chosen option B because the ...,10.333605,You are an AI assistant acting as a healthcare...
4,https://jamanetwork.com/journals/jamaotolaryng...,A 19-year-old woman with no noteworthy medical...,Dentigerous cyst,Odontogenic keratocyst (keratocystic odontogen...,Ameloblastoma,Adenomatoid odontogenic tumor,D. Adenomatoid odontogenic tumor,D,Adenomatoid odontogenic tumor,Adenomatoid odontogenic tumors are uncommon od...,Clinical Challenge,What Is Your Diagnosis?,female,296,original,<answer>\nB\nI've chosen option B because the ...,11.099002,You are an AI assistant acting as a healthcare...


In [3]:
df.columns


Index(['link', 'case', 'opa', 'opb', 'opc', 'opd', 'diagnosis', 'answer_idx',
       'answer', 'explanation', 'field', 'clinical_question', 'gender',
       'case_id', 'version', 'llm_gpt3_response', 'llm_gpt3_time',
       'llm_gpt3_prompt'],
      dtype='object')

In [4]:
print(df['answer_idx'][0])


C


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   link               90 non-null     object 
 1   case               90 non-null     object 
 2   opa                90 non-null     object 
 3   opb                90 non-null     object 
 4   opc                90 non-null     object 
 5   opd                90 non-null     object 
 6   diagnosis          87 non-null     object 
 7   answer_idx         90 non-null     object 
 8   answer             90 non-null     object 
 9   explanation        90 non-null     object 
 10  field              90 non-null     object 
 11  clinical_question  90 non-null     object 
 12  gender             90 non-null     object 
 13  case_id            90 non-null     int64  
 14  version            90 non-null     object 
 15  llm_gpt3_response  90 non-null     object 
 16  llm_gpt3_time      90 non-nu

In [6]:
import pandas as pd

# Assuming your DataFrame is called 'df'

# 1. Create llm_gpt3_choice column
df['llm_gpt3_choice'] = df['llm_gpt3_response'].str.split('<answer>\n', expand=True)[1].str[0]

# 2. Create llm_gpt3_discussion column
df['llm_gpt3_discussion'] = df['llm_gpt3_response'].str.split('\n', expand=True)[1]

# 3. Calculate accuracy
# Function to calculate accuracy
def calculate_accuracy(group):
    return (group['llm_gpt3_choice'] == group['answer_idx']).mean()

# Calculate accuracy for each case_id, gender, and version
accuracy_by_case = df.groupby('case_id', as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
accuracy_by_case['category'] = 'By Case ID'

accuracy_by_gender = df.groupby('gender', as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
accuracy_by_gender['category'] = 'By Gender'

accuracy_by_version = df.groupby('version', as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
accuracy_by_version['category'] = 'By Version'

accuracy_by_gender_version = df.groupby(['gender', 'version'], as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
accuracy_by_gender_version['category'] = 'By Gender and Version'

overall_accuracy = pd.DataFrame({'category': ['Overall'], 'accuracy': [calculate_accuracy(df)]})

# Combine all results into a single table
results = pd.concat([
    accuracy_by_case,
    accuracy_by_gender,
    accuracy_by_version,
    accuracy_by_gender_version,
    overall_accuracy
], sort=False)

# Reorder columns
results = results[['category'] + [col for col in results.columns if col != 'category' and col != 'accuracy'] + ['accuracy']]

# Sort the results
results = results.sort_values(['category', 'gender', 'version', 'case_id']).reset_index(drop=True)

# Fill NaN values with empty string for better display
results = results.fillna('')

# Display the table
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.2f}'.format)
# print(results.to_string(index=False))


/var/folders/s2/jdv7kxv13hgf_3qqk7bg5p3r0000gn/T/ipykernel_2946/454238519.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_case = df.groupby('case_id', as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
/var/folders/s2/jdv7kxv13hgf_3qqk7bg5p3r0000gn/T/ipykernel_2946/454238519.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_gender = df.groupby('ge

In [7]:
results


,category,case_id,gender,version,accuracy
0,By Case ID,49.00,,,0.00
1,By Case ID,74.00,,,0.00
2,By Case ID,95.00,,,0.00
3,By Case ID,131.00,,,0.00
4,By Case ID,162.00,,,0.00
5,By Case ID,164.00,,,1.00
6,By Case ID,296.00,,,0.00
7,By Case ID,301.00,,,0.00
8,By Case ID,313.00,,,0.00
9,By Case ID,316.00,,,0.00
